In [56]:
import requests
import time
import getpass

# Pedimos el token sin mostrarlo en pantalla
GITHUB_TOKEN = getpass.getpass("Ingresa tu GitHub token: ")




Ingresa tu GitHub token: ··········


In [69]:
import requests
import time

class GitHubApiClient:
    def __init__(self, token):
        self.base_url = "https://api.github.com"
        self.headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/vnd.github.v3+json"
        }

    def _handle_rate_limit(self, response):
        if response.status_code == 403 and 'X-RateLimit-Remaining' in response.headers:
            remaining = int(response.headers.get('X-RateLimit-Remaining', 1))
            if remaining == 0:
                reset_time = int(response.headers.get('X-RateLimit-Reset', 0))
                wait_seconds = max(reset_time - time.time(), 0) + 1
                print(f"Rate limit alcanzado. Esperando {int(wait_seconds)} segundos...")
                time.sleep(wait_seconds)
                return True
        return False

    def _get(self, endpoint, params=None):
        url = f"{self.base_url}{endpoint}"
        while True:
            response = requests.get(url, headers=self.headers, params=params)
            if self._handle_rate_limit(response):
                continue
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error {response.status_code} en {url}: {response.text}")
                return None

    def get_all_pages(self, endpoint, params=None):
        all_data = []
        page = 1
        per_page = 100
        while True:
            params_page = params.copy() if params else {}
            params_page.update({"page": page, "per_page": per_page})
            data = self._get(endpoint, params=params_page)
            if not data:
                break
            all_data.extend(data)
            if len(data) < per_page:
                break
            page += 1
        return all_data

    def search_repositories(self, query, sort="stars", order="desc"):
        endpoint = "/search/repositories"
        params = {"q": query, "sort": sort, "order": order}
        result = self._get(endpoint, params=params)
        if result:
            return result.get("items", [])
        return []

    def list_commits(self, owner, repo):
        endpoint = f"/repos/{owner}/{repo}/commits"
        return self.get_all_pages(endpoint)

    def get_repo_contents(self, owner, repo, path=""):
        endpoint = f"/repos/{owner}/{repo}/contents/{path}"
        return self._get(endpoint)




In [65]:
client = GitHubApiClient(GITHUB_TOKEN)

In [66]:
repos = client.search_repositories("data source")
print(f"Repositorios encontrados: {len(repos)}")
for r in repos[:5]:
    print(f"- {r['full_name']} ⭐ {r['stargazers_count']}")


Repositorios encontrados: 30
- grafana/grafana ⭐ 68929
- AppFlowy-IO/AppFlowy ⭐ 64356
- metabase/metabase ⭐ 42723
- AykutSarac/jsoncrack.com ⭐ 39372
- getredash/redash ⭐ 27519


In [67]:
if repos:
    owner, repo = repos[0]['owner']['login'], repos[0]['name']
    print(f"Obteniendo commits de {owner}/{repo} (limitado a 30)...")
    try:
        # Hacemos una llamada simple para traer solo los primeros 30 commits
        endpoint = f"/repos/{owner}/{repo}/commits"
        params = {"per_page": 30}
        commits = client._get(endpoint, params=params)  # Usamos método interno para no paginar todo
        print(f"Total commits obtenidos: {len(commits)}")
        for c in commits[:3]:
            print(f"- {c['commit']['message']}")
    except Exception as e:
        print("❌ ERROR al obtener commits:", e)
else:
    print("No hay repositorios para listar commits.")

Obteniendo commits de grafana/grafana (limitado a 30)...
Total commits obtenidos: 30
- docs: Clarifying the support level of SCIM (#108034)
- Chore: Fix releaseFinder script perf (#108035)

* baldm0mma/ speed up branch loop

* baldm0mma/ rem unneeded comment

* baldm0mma/ update release output format

* baldm0mma/ refactor
- Secrets: Bump API version to v1beta1 (#108026)


In [68]:
if repos:
    owner, repo = repos[0]['owner']['login'], repos[0]['name']
    contents = client.get_repo_contents(owner, repo)
    if contents:
        print(f"Contenido raíz de {owner}/{repo}:")
        for item in contents:
            print(f"- {item['type']} {item['name']}")


Contenido raíz de grafana/grafana:
- file .air.toml
- file .betterer.eslint.config.js
- file .betterer.results
- file .betterer.ts
- dir .bingo
- file .bra.toml
- file .browserslistrc
- dir .changelog-archive
- dir .citools
- file .dockerignore
- file .drone.star
- file .drone.yml
- file .editorconfig
- file .gitattributes
- dir .github
- file .gitignore
- file .golangci.yml
- dir .husky
- file .ignore
- file .levignore.js
- file .nvmrc
- file .nxignore
- file .prettierignore
- file .prettierrc.js
- file .trivyignore
- file .vale.ini
- dir .vim
- dir .vscode
- dir .yarn
- file .yarnrc.yml
- file CHANGELOG.md
- file CODE_OF_CONDUCT.md
- file CONTRIBUTING.md
- file Dockerfile
- file GOVERNANCE.md
- file HALL_OF_FAME.md
- file LICENSE
- file LICENSING.md
- file MAINTAINERS.md
- file Makefile
- file NOTICE.md
- file README.md
- file ROADMAP.md
- file SECURITY.md
- file SUPPORT.md
- file WORKFLOW.md
- dir apps
- file build.go
- dir conf
- dir contribute
- file crowdin.yml
- dir cue.mod
- fi